In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats.mstats import winsorize
from scipy.stats import yeojohnson
import warnings
warnings.filterwarnings("ignore")

In [2]:
DATA_DIR = r"c:\Users\nicta\Desktop\API_Paper_ChiPlay\mod_data"

In [3]:
# ── 1. Load parquet files ──────────────────────────────────────────────────────
print("Loading parquet files...")
authors = pd.read_parquet(f"{DATA_DIR}/Authors.parquet")
mods    = pd.read_parquet(f"{DATA_DIR}/CleanedModData.parquet")
cats    = pd.read_parquet(f"{DATA_DIR}/GameCategories.parquet")

Loading parquet files...


In [4]:
print(f"  Authors:        {len(authors):,} rows")
print(f"  CleanedModData: {len(mods):,} rows")
print(f"  GameCategories: {len(cats):,} rows")

  Authors:        122,859 rows
  CleanedModData: 483,939 rows
  GameCategories: 5,089 rows


In [5]:
# Filter authors: deleted=0, last_active >= 2024-01-01
# Note: 'deleted' column is bool in parquet
authors_filtered = authors[
    (~authors["deleted"]) &
    (authors["last_active"].notna()) &
    (authors["last_active"] >= "2024-01-01")
].copy()
print(f"  Authors after filter: {len(authors_filtered):,}")

  Authors after filter: 73,855


In [6]:
# Cast bool to int for aggregation
mods["contains_adult_content"] = mods["contains_adult_content"].astype(int)

In [7]:
# Join mods with GameCategories to get new_group_category
mods_with_cats = mods.merge(
    cats[["game_id", "category_id", "new_group_category"]],
    on=["game_id", "category_id"],
    how="left"
)

In [8]:
# Aggregate mods to user level
mod_agg = mods_with_cats.groupby("member_id").agg(
    first_mod_created_date=("created_timestamp", "min"),
    last_mod_created_date=("created_timestamp", "max"),
    total_domains=("domain_name", "nunique"),
    total_categories=("new_group_category", "nunique"),
    endorsements_received=("endorsement_count", "count"),
    adult_content_count=("contains_adult_content", "sum"),
    all_mod_downloads=("mod_downloads", "sum"),
    all_unique_mod_downloads=("mod_unique_downloads", "sum"),
).reset_index()

In [9]:
# Join authors with mod aggregates
df = authors_filtered.merge(mod_agg, on="member_id", how="left")

In [10]:
# Compute derived columns matching the SQL query
df["active_days"] = (df["last_active"] - df["joined"]).dt.days
df["published_mod_count"] = df["mod_count"]
df["unpublished_mod_count"] = df["owned_mod_count"] - df["mod_count"]
df["all_mods_count"] = df["owned_mod_count"]

In [11]:
# Convert unix timestamps to datetime for mod dates
df["first_mod_created_date_dt"] = pd.to_datetime(df["first_mod_created_date"], unit="s", errors="coerce")
df["last_mod_created_date_dt"] = pd.to_datetime(df["last_mod_created_date"], unit="s", errors="coerce")

In [12]:
# Compute mod_creation_days_since_joined (days from join to first mod)
df["mod_creation_days_since_joined"] = (df["first_mod_created_date_dt"] - df["joined"]).dt.days

In [14]:
# Select the 13 features used in the Devotion Score
feature_cols = [
    "last_mod_created_date",
    "published_mod_count",
    "unpublished_mod_count",
    "all_mods_count",
    "endorsements_given",
    "posts",
    "kudos",
    "views",
    "endorsements_received",
    "adult_content_count",
    "all_mod_downloads",
    "all_unique_mod_downloads",
    "mod_creation_days_since_joined",
]

In [15]:
df_numeric = df[feature_cols].copy().fillna(0)

In [16]:
df_transformed = df_numeric.copy()

In [17]:
# Log1p transforms (clip negatives to 0 first — a few rows have negative posts)
for col in ["endorsements_given", "posts", "kudos"]:
    df_transformed[col] = np.log1p(df_transformed[col].clip(lower=0))

In [18]:
# Winsorize + cube-root for views
df_transformed["views"] = winsorize(df_transformed["views"], limits=[0.02, 0.02])
df_transformed["views"] = np.cbrt(df_transformed["views"])

In [19]:
# Yeo-Johnson transforms
yj_cols = [
    "published_mod_count", "unpublished_mod_count", "all_mods_count",
    "endorsements_received", "adult_content_count",
    "all_mod_downloads", "all_unique_mod_downloads",
]
for col in yj_cols:
    df_transformed[col], _ = yeojohnson(df_transformed[col])

In [20]:
# Z-standardise
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_transformed)

In [21]:
print(f"  Scaled data shape: {data_scaled.shape}")

  Scaled data shape: (73855, 13)


In [22]:
pca = PCA(n_components=7)
pca_data = pca.fit_transform(data_scaled)

In [23]:
print("  Explained variance ratios:", np.round(pca.explained_variance_ratio_, 4))
print(f"  Total variance explained: {pca.explained_variance_ratio_.sum():.4f}")

  Explained variance ratios: [0.4429 0.1333 0.0944 0.0931 0.0731 0.0463 0.0436]
  Total variance explained: 0.9267


In [26]:
# ── 6. Devotion Score ──────────────────────────────────────────────────────────
loadings = np.abs(pca.components_)
explained_variance = pca.explained_variance_ratio_
feature_weights = np.dot(explained_variance, loadings)
devotion_scores = np.dot(data_scaled, feature_weights)

In [27]:
df["Devotion_Score"] = devotion_scores
print(f"  Devotion Score - Mean: {devotion_scores.mean():.4f}, "
      f"Median: {np.median(devotion_scores):.4f}, "
      f"Std: {devotion_scores.std():.4f}")

  Devotion Score - Mean: 0.0000, Median: -0.3173, Std: 1.7129


In [28]:
# ── 7. K-Means Clustering ─────────────────────────────────────────────────────
print("\nRunning K-Means clustering (k=4)...")
kmeans = KMeans(n_clusters=4, n_init=50, random_state=42)
df["Cluster"] = kmeans.fit_predict(pca_data)


Running K-Means clustering (k=4)...


  File "C:\Users\nicta\miniconda3\envs\data_analysis_conda\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\nicta\miniconda3\envs\data_analysis_conda\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nicta\miniconda3\envs\data_analysis_conda\Lib\subprocess.py", line 1036, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                        pass_fds, cwd, env,
                        ^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
                        gid, gids, uid, umask,
                        ^^^^^^^^^^^^^^^^^^^^^^
                        start_new_session, process_group)
                        ^^^^

In [29]:
# Sort clusters by mean Devotion Score so labelling is consistent
cluster_means = df.groupby("Cluster")["Devotion_Score"].mean().sort_values()
cluster_map = {old: new for new, old in enumerate(cluster_means.index)}
df["Devotion_Cluster"] = df["Cluster"].map(cluster_map)

In [30]:
print("  Cluster sizes:")
for c in sorted(df["Devotion_Cluster"].unique()):
    n = (df["Devotion_Cluster"] == c).sum()
    pct = n / len(df) * 100
    mean_d = df.loc[df["Devotion_Cluster"] == c, "Devotion_Score"].mean()
    print(f"    Cluster {c}: {n:,} ({pct:.1f}%), mean Devotion = {mean_d:.2f}")

  Cluster sizes:
    Cluster 0: 14,437 (19.5%), mean Devotion = -1.59
    Cluster 1: 27,903 (37.8%), mean Devotion = -0.97
    Cluster 2: 21,519 (29.1%), mean Devotion = 0.87
    Cluster 3: 9,996 (13.5%), mean Devotion = 3.13


In [32]:
# Activity span = last_mod - first_mod (in days)
df["activity_span_days"] = (
    df["last_mod_created_date_dt"] - df["first_mod_created_date_dt"]
).dt.days
df["activity_span_years"] = df["activity_span_days"] / 365.25

In [33]:
print("\n-- Original computation (for comparison) --")
df_orig = df.copy()
df_orig["activity_span_years_orig"] = df_orig["activity_span_years"].replace(0, np.nan)
df_orig["mods_per_year_orig"] = df_orig["all_mods_count"] / df_orig["activity_span_years_orig"]


-- Original computation (for comparison) --


In [34]:
orig_stats = df_orig.groupby("Devotion_Cluster").agg(
    span_days_mean=("activity_span_days", "mean"),
    span_days_median=("activity_span_days", "median"),
    span_years_mean=("activity_span_years", "mean"),
    mods_per_year_mean=("mods_per_year_orig", "mean"),
).round(2)
print(orig_stats.to_string())
print("\n** Note: mods_per_year_mean is inflated by users with tiny spans")

                  span_days_mean  span_days_median  span_years_mean  mods_per_year_mean
Devotion_Cluster                                                                       
0                          52.05               0.0             0.14              128.86
1                          32.75               0.0             0.09              139.48
2                         519.73             154.0             1.42              107.09
3                        1278.75             897.0             3.50               67.86

** Note: mods_per_year_mean is inflated by users with tiny spans


In [37]:
df["activity_span_years_floored"] = np.maximum(df["activity_span_days"], 365) / 365.25
df["mods_per_year_floored"] = df["all_mods_count"] / df["activity_span_years_floored"]

In [38]:
floored_stats = df.groupby("Devotion_Cluster").agg(
    span_days_mean=("activity_span_days", "mean"),
    span_days_median=("activity_span_days", "median"),
    mods_per_year_floored_mean=("mods_per_year_floored", "mean"),
    mods_per_year_floored_median=("mods_per_year_floored", "median"),
).round(2)
print(floored_stats.to_string())

                  span_days_mean  span_days_median  mods_per_year_floored_mean  mods_per_year_floored_median
Devotion_Cluster                                                                                            
0                          52.05               0.0                        1.63                          1.00
1                          32.75               0.0                        1.26                          1.00
2                         519.73             154.0                        4.47                          3.00
3                        1278.75             897.0                       11.85                          5.01


In [39]:
df_30plus = df[df["activity_span_days"] >= 30].copy()
df_30plus["mods_per_year_clean"] = df_30plus["all_mods_count"] / df_30plus["activity_span_years"]

In [40]:
clean_stats = df_30plus.groupby("Devotion_Cluster").agg(
    n_users=("member_id", "count"),
    span_days_mean=("activity_span_days", "mean"),
    span_days_median=("activity_span_days", "median"),
    mods_per_year_mean=("mods_per_year_clean", "mean"),
    mods_per_year_median=("mods_per_year_clean", "median"),
    total_mods_mean=("all_mods_count", "mean"),
).round(2)
print(clean_stats.to_string())

                  n_users  span_days_mean  span_days_median  mods_per_year_mean  mods_per_year_median  total_mods_mean
Devotion_Cluster                                                                                                      
0                    1344          554.37             303.5                9.12                  4.01             4.27
1                    1687          534.15             268.0                5.85                  2.86             2.12
2                   13937          799.32             445.0               11.45                  3.93             6.20
3                    9243         1382.28            1018.0               19.46                  5.62            26.42


In [41]:
print("\n-- Active days (joined -> last_active) per cluster --")
active_days_stats = df.groupby("Devotion_Cluster").agg(
    active_days_mean=("active_days", "mean"),
    active_days_median=("active_days", "median"),
).round(0)
print(active_days_stats.to_string())


-- Active days (joined -> last_active) per cluster --
                  active_days_mean  active_days_median
Devotion_Cluster                                      
0                           2861.0              2636.0
1                           3488.0              3639.0
2                           3506.0              3585.0
3                           3904.0              4061.0


In [42]:
print("\n1. APPROACH A -- Floor span at 1 year for rate calculation:")
print("   This avoids division-by-tiny-numbers while keeping all users.")
print(floored_stats[["mods_per_year_floored_mean", "mods_per_year_floored_median"]].to_string())


1. APPROACH A -- Floor span at 1 year for rate calculation:
   This avoids division-by-tiny-numbers while keeping all users.
                  mods_per_year_floored_mean  mods_per_year_floored_median
Devotion_Cluster                                                          
0                                       1.63                          1.00
1                                       1.26                          1.00
2                                       4.47                          3.00
3                                      11.85                          5.01


In [43]:
print("\n2. APPROACH B — Exclude users with span < 30 days:")
print("   More honest, but reduces sample size significantly for some clusters.")
print(clean_stats[["n_users", "mods_per_year_mean", "mods_per_year_median"]].to_string())


2. APPROACH B — Exclude users with span < 30 days:
   More honest, but reduces sample size significantly for some clusters.
                  n_users  mods_per_year_mean  mods_per_year_median
Devotion_Cluster                                                   
0                    1344                9.12                  4.01
1                    1687                5.85                  2.86
2                   13937               11.45                  3.93
3                    9243               19.46                  5.62


In [44]:
print("\n3. Active days (joined -> last_active) -- reliable, use in appendix table:")
print(active_days_stats.to_string())


3. Active days (joined -> last_active) -- reliable, use in appendix table:
                  active_days_mean  active_days_median
Devotion_Cluster                                      
0                           2861.0              2636.0
1                           3488.0              3639.0
2                           3506.0              3585.0
3                           3904.0              4061.0


In [46]:
skew_before = df_numeric.skew().round(2)
skew_after = pd.DataFrame(data_scaled, columns=feature_cols).skew().round(2)

In [47]:
skew_table = pd.DataFrame({
    "Feature": feature_cols,
    "Raw_Skewness": skew_before.values,
    "Post_Transform_Skewness": skew_after.values,
    "Transform": [
        "None",            # last_mod_created_date
        "Yeo-Johnson",     # published_mod_count
        "Yeo-Johnson",     # unpublished_mod_count
        "Yeo-Johnson",     # all_mods_count
        "log1p",           # endorsements_given
        "log1p",           # posts
        "log1p",           # kudos
        "Winsorize+cbrt",  # views
        "Yeo-Johnson",     # endorsements_received
        "Yeo-Johnson",     # adult_content_count
        "Yeo-Johnson",     # all_mod_downloads
        "Yeo-Johnson",     # all_unique_mod_downloads
        "None",            # mod_creation_days_since_joined
    ]
})
print(skew_table.to_string(index=False))

                       Feature  Raw_Skewness  Post_Transform_Skewness      Transform
         last_mod_created_date         -0.90                    -0.90           None
           published_mod_count         22.75                     0.08    Yeo-Johnson
         unpublished_mod_count         36.88                     0.46    Yeo-Johnson
                all_mods_count         22.04                     0.39    Yeo-Johnson
            endorsements_given          9.75                     0.21          log1p
                         posts         32.69                     0.29          log1p
                         kudos         34.39                     1.28          log1p
                         views         45.31                     1.12 Winsorize+cbrt
         endorsements_received         26.12                     0.50    Yeo-Johnson
           adult_content_count        103.29                     2.18    Yeo-Johnson
             all_mod_downloads         38.01                    -